In [1]:
import pandas as pd
import re
import json
import requests
from transformers import pipeline
from time import sleep

import pandas as pd
import re
import json
import requests
import chardet
from transformers import pipeline
from unicodedata import category

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read(10000)
    return chardet.detect(rawdata)['encoding']

def remove_control_chars(text):
    return ''.join(c for c in str(text) if category(c)[0] != 'C')

# Загрузка данных с автоматическим определением кодировки
file_path = '/home/jupyter/datasphere/project/ver4/cluster_centers/longformer_hdbscan_centers.csv'

try:
    # Определение кодировки
    encoding = detect_encoding(file_path)
    
    # Чтение файла с обработкой ошибок
    df = pd.read_csv(
        file_path,
        encoding=encoding,
        on_bad_lines='warn',
        engine='python',
        dtype={'error_text': str}
    )
    
    # Проверка обязательных колонок
    required_columns = ['cluster', 'log_id', 'error_text']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"CSV file must contain columns: {required_columns}")

except UnicodeDecodeError:
    print("Ошибка кодировки. Пробуем fallback кодировки...")
    try:
        df = pd.read_csv(file_path, encoding='latin-1', dtype={'error_text': str})
    except Exception as e:
        print(f"Не удалось прочитать файл: {str(e)}")
        exit()

# Функция очистки текста
def clean_text(text):
    # Удаление управляющих символов
    cleaned = remove_control_chars(text)
    # Удаление начальных цифр
    cleaned = re.sub(r'^\d+', '', cleaned)
    # Удаление фиксированного окончания
    return cleaned.replace('Name: error_text, dtype: object', '').strip()

# Применение очистки с фильтрацией битых строк
df['cleaned_text'] = df['error_text'].apply(clean_text)
df = df[df['cleaned_text'].str.len() > 5]

grouped = df.groupby('cluster')['cleaned_text'].agg(' '.join).reset_index()

# Инициализация модели для суммаризации
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Функция суммаризации с обработкой длинных текстов
def summarize_text(text):
    truncated = text[:1024]
    print(f"СУММАРИЗАЦИЯ ТЕКСТА: {text}")
    result = summarizer(truncated, max_length=1024, min_length=30, do_sample=False)
    return result[0]['summary_text']

# Применение суммаризации
grouped['model_summary'] = grouped['cleaned_text'].apply(summarize_text)

# Подготовка данных для DeepSeek
cluster_summaries = [f"{row['cluster']}: {row['model_summary']}" for _, row in grouped.iterrows()]
batch_prompt = "\n".join(cluster_summaries)

# Конфигурация DeepSeek API
DEEPSEEK_API_KEY = "sk-caf1d8be739f4e2ea2da177bd37e5c01"
API_URL = "https://api.deepseek.com/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
    "Content-Type": "application/json"
}

# Формирование запроса
payload = {
    "model": "deepseek-chat",
    "messages": [{
        "role": "user",
        "content": f"""Проанализируй краткие описания ошибок и создай для каждой уникальное название на английском языке.
Требования:
1. Только 1-3 технических слова через пробел
2. Все названия должны быть уникальными
3. Формат ответа ТОЛЬКО как в примере:

0: ConnectionTimeout
1: InvalidJSONFormat
...
 
Вот описания ошибок:
{batch_prompt}"""
    }],
    "temperature": 0.3
}

# Отправка запроса и обработка ответа
result_dict = {}
try:
    response = requests.post(API_URL, json=payload, headers=headers)
    if response.status_code == 200:
        raw_response = response.json()['choices'][0]['message']['content']
        # Парсинг ответа
        for line in raw_response.strip().split('\n'):
            if ':' in line:
                cluster_id, error_name = line.split(':', 1)
                result_dict[cluster_id.strip()] = error_name.strip()
    else:
        print(f"API Error: {response.status_code} - {response.text}")
        exit()
except Exception as e:
    print(f"Request failed: {str(e)}")
    exit()

# Формирование финального результата
final_output = {}
for _, row in grouped.iterrows():
    cluster = str(row['cluster'])
    final_output[cluster] = {
        "model_summary": row['model_summary'],
        "deepseek_name": result_dict.get(cluster, "UnknownError")
    }

# Сохранение результатов
with open('cluster_names_v2.json', 'w') as f:
    json.dump(final_output, f, indent=2, ensure_ascii=False)

print("Обработка завершена. Результаты сохранены в cluster_names_v2.json")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-05-18 01:37:55.788775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-18 01:38:06.758625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Device set to use cuda:0
Your max_length is set to 1024, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)


СУММАРИЗАЦИЯ ТЕКСТА: Package 'bzip2', required by 'freetype2', not ... Package bzip2 was not found in the pkg-config ... Using default compression for debug info becau... -- Looking for backtrace Problem opening /proc/meminfo Problem opening /proc/meminfo Problem opening /proc/meminfo 2817 |         while (currentLine < endLine ||... 607 |             if (breakpoint >= aEnd) { -- Looking for sys/types.h - found


Your max_length is set to 1024, but your input_length is only 123. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


СУММАРИЗАЦИЯ ТЕКСТА: checking for wsncpy... no checking for main in -lXmu... yes Making all in app-defaults In file included from DisplayCAL/RealDisplaySi... + find . -name '*.so' -print -delete Hunk #5 succeeded at 19537 (offset 52 lines). 2697 |     unsigned long*      /* bytes_after_... 724 |     unsigned long interval; Hunk #4 succeeded at 3272 (offset 15 lines). Hunk #3 succeeded at 3133 (offset 15 lines).


Your max_length is set to 1024, but your input_length is only 173. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=86)


...

СУММАРИЗАЦИЯ ТЕКСТА: checking for connect... yes checking for dlltool... no 159 |     if (extContext == (XContext)NULL) ex... checking for x86_64-alt-linux-mt... no checking for wslen... no 1068 |             XtCallCallbacks (w, XtNscro... -- Detecting C compiler ABI info - done make[2]: *** [Makefile:552: laylex.lo] Error 1 checking for main in -lXmu... yes Package 'libidn2', required by 'libcurl', not ...


Your max_length is set to 1024, but your input_length is only 146. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=73)


СУММАРИЗАЦИЯ ТЕКСТА: <86>May 17 05:03:02 groupadd[725610]: group ad... to the PKG_CONFIG_PATH environment variable -- Performing Test COMPILER_HAS_DEPRECATED_ATTR -- Check for working CXX compiler: /usr/bin/c+... checking for x86_64-alt-linux-gcc option to en... Problem opening /proc/meminfo Package libidn2 was not found in the pkg-confi... -- Could NOT find BZip2 (missing: BZIP2_LIBRAR... checking for x86_64-alt-linux-ar... no Problem opening /proc/meminfo


Your max_length is set to 1024, but your input_length is only 176. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=88)


СУММАРИЗАЦИЯ ТЕКСТА: [ 37%] Built target imgui_implot 131 |                 return _pos == _data.siz... [ 18%] Built target imgui_implot3d /usr/bin/ranlib libminiaudio.a /usr/src/RPM/BUILD/apache-maven-3.8.8/maven-re... 46 |         bool m_localOnly = true; 83 |                 UInt32 sum = 0, index = 0; /usr/bin/ranlib libromfs-builtin.a [ 31%] Built target miniaudio [ 34%] Built target libromfs-script_loader
Обработка завершена. Результаты сохранены в cluster_names_v2.json


In [1]:
import pandas as pd
import re
import json
import requests
from transformers import pipeline

df = pd.read_csv('/home/jupyter/datasphere/project/ver4/cluster_centers/longformer_hdbscan_centers.csv')

def clean_text(text):
    text = re.sub(r'^\d+', '', text)
    text = text.replace('Name: error_text, dtype: object', '').strip()
    return text

df['cleaned_text'] = df['error_text'].apply(clean_text)

grouped = df.groupby('cluster')['cleaned_text'].agg(' '.join).reset_index()

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def summarize_text(text):
    inputs = text[:1024]
    result = summarizer(inputs, max_length=75, min_length=30, do_sample=False)
    return result[0]['summary_text']

grouped['summary'] = grouped['cleaned_text'].apply(summarize_text)

DEEPSEEK_API_KEY = "sk-caf1d8be739f4e2ea2da177bd37e5c01"
API_URL = "https://api.deepseek.com/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
    "Content-Type": "application/json"
}

def get_error_name(summary):
    payload = {
        "model": "deepseek-chat",
        "messages": [
            {
                "role": "user",
                "content": f"Сгенерируй краткое техническое название этой ошибки на английском языке без лишних слов, только суть: {summary}"
            }
        ]
    }
    
    response = requests.post(API_URL, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content'].strip()
    return "Unknown Error"

result = {}
for _, row in grouped.iterrows():
    error_name = get_error_name(row['summary'])
    result[row['cluster']] = error_name

with open('cluster_names.json', 'w') as f:
    json.dump(result, f, indent=2)

print("Обработка завершена. Результаты сохранены в cluster_names.json")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-05-18 00:47:34.344677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-18 00:47:42.537118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Device set to use cuda:0
Your max_length is set to 75, but your input_length is only 41. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)
Your max_length is set to 75, but your input_length is only 27. Since t

Обработка завершена. Результаты сохранены в cluster_names.json
